In [1]:
import nltk

In [2]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [3]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [4]:
import kagglehub
import pandas as pd

In [5]:
path = r'C:\Users\david\.cache\kagglehub\datasets\abhishekunnam\meeting-transcripts\versions\1'

In [6]:
train_data = pd.read_csv(f'{path}/train_df.csv')

In [7]:
train_data.head(3)

,id,Item_UID,Meeting_UID,Transcript,Summary,Date
0,DenverCityCouncil_01062014_13-0757,13-0757,DenverCityCouncil_01062014,"Speaker 3: We reconvened, we have one more req...",Approves changes to the zoning classification ...,2014-01-06
1,DenverCityCouncil_01062014_13-1022,13-1022,DenverCityCouncil_01062014,"Speaker 3: And so that's a great story, Counci...",A proclamation welcoming the 108th National We...,2014-01-06
2,LongBeachCC_01072014_14-0021,14-0021,LongBeachCC_01072014,Speaker 2: 23 and 23 is a recommended recommen...,Recommendation to declare ordinance amending t...,2014-01-07


In [8]:
Transcript_Data = train_data[["Transcript", "Summary"]]

In [9]:
Transcript_Data = Transcript_Data.dropna()

In [10]:
Transcript_Data.head(3)

,Transcript,Summary
0,"Speaker 3: We reconvened, we have one more req...",Approves changes to the zoning classification ...
1,"Speaker 3: And so that's a great story, Counci...",A proclamation welcoming the 108th National We...
2,Speaker 2: 23 and 23 is a recommended recommen...,Recommendation to declare ordinance amending t...


In [11]:
Transcript_Data['Transcript'] = Transcript_Data['Transcript'].str.replace(r"Speaker \d+:", "", regex=True).str.replace(r"\s+", " ", regex=True).str.strip()

In [12]:
Transcript_Data.isna().sum()

Transcript    0
Summary       0
dtype: int64

In [13]:
Transcript_Data["input_text"] =  "summarize this for an interview: " + Transcript_Data["Transcript"]

Transcript_Data["target_text"] = Transcript_Data["Summary"]

In [14]:
Transcript_Data

,Transcript,Summary,input_text,target_text
0,"We reconvened, we have one more required publi...",Approves changes to the zoning classification ...,summarize this for an interview: We reconvened...,Approves changes to the zoning classification ...
1,"And so that's a great story, Councilman. Thank...",A proclamation welcoming the 108th National We...,summarize this for an interview: And so that's...,A proclamation welcoming the 108th National We...
2,23 and 23 is a recommended recommendation to d...,Recommendation to declare ordinance amending t...,summarize this for an interview: 23 and 23 is ...,Recommendation to declare ordinance amending t...
3,Item 13 is a report from the office Councilwom...,Recommendation to request City Manager to prov...,summarize this for an interview: Item 13 is a ...,Recommendation to request City Manager to prov...
4,Okay. Now we go to item 15 luxury. And 15 is a...,Recommendation to adopt resolution Version A o...,summarize this for an interview: Okay. Now we ...,Recommendation to adopt resolution Version A o...
...,...,...,...,...
5164,NPA submits a report recommending that the rep...,"On the message and order, referred on July 13,...",summarize this for an interview: NPA submits a...,"On the message and order, referred on July 13,..."
5165,"Mr. Kirk, please. RE Docket 0965. Duncan, Emma...",Order for a hearing on increasing Green Stormw...,"summarize this for an interview: Mr. Kirk, ple...",Order for a hearing on increasing Green Stormw...
5166,"Docket 0967 has been adopted. Mr. Clarke, if y...",Resolution Recognizing August 7 - 13 As Nation...,summarize this for an interview: Docket 0967 h...,Resolution Recognizing August 7 - 13 As Nation...
5167,"Okay, thank you very much. See no further requ...",A MOTION confirming the appointment of Allen N...,"summarize this for an interview: Okay, thank y...",A MOTION confirming the appointment of Allen N...


In [17]:
df = Transcript_Data[['input_text','target_text']]

In [18]:
df.rename(columns = {
    'input_text':'Transcript',
    'target_text':'Summary'
}, inplace=True)
          

C:\Users\david\AppData\Local\Temp\ipykernel_12632\3772444807.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {


In [25]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load model with eager attention
model = AutoModelForSeq2SeqLM.from_pretrained(
    "t5-small",
    attn_implementation="eager"
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained("t5-small")

# Example input
inputs = tokenizer("This is a test", return_tensors="pt")

# Optional for decoder
decoder_input_ids = torch.ones((1, 1), dtype=torch.long)

# Export to ONNX
torch.onnx.export(
    model,
    (inputs["input_ids"], inputs["attention_mask"], decoder_input_ids),
    "tiny_summarizer.onnx",
    input_names=["input_ids", "attention_mask", "decoder_input_ids"],
    output_names=["logits"],
    opset_version=11,
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "sequence_length"},
        "attention_mask": {0: "batch_size", 1: "sequence_length"},
        "decoder_input_ids": {0: "batch_size", 1: "sequence_length"},
        "logits": {0: "batch_size", 1: "sequence_length"}
    }
)

print("ONNX model exported successfully!")


C:\Users\david\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\david\anaconda3\Lib\site-packages\transformers\modeling_utils.py:943: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if causal_mask.shape[1] < attention_mask.shape[1]:


ONNX model exported successfully!


C:\Users\david\anaconda3\Lib\site-packages\torch\onnx\utils.py:1703: UserWarning: The exported ONNX model failed ONNX shape inference. The model will not be executable by the ONNX Runtime. If this is unintended and you believe there is a bug, please report an issue at https://github.com/pytorch/pytorch/issues. Error reported by strict ONNX shape inference: [ShapeInferenceError] (op_type:Min, node name: /encoder/block.0/layer.0/SelfAttention/Min): data_0 typestr: T, has unsupported type: tensor(int64) (Triggered internally at ..\torch\csrc\jit\serialization\export.cpp:1490.)
  _C._check_onnx_proto(proto)


In [28]:
from onnxruntime.quantization import quantize_dynamic, QuantType
import os

quantize_dynamic(
    "t5_summarizer.onnx",
    "t5_summarizer_quant.onnx",
    weight_type=QuantType.QInt8
)

print("Quantized size (MB):", os.path.getsize("t5_summarizer_quant.onnx") / 1e6)

Quantized size (MB): 77.463535
